# Using EDP Symbology Conversion Service with Intraday Summaries serivice

## Introduction
To retrieve intraday summaries data i.e. bar data, you need to specify a Refinitiv RIC of the data. This sample demonstrates how to use EDP Symbology Conversion Service to convert two symbols e.g.ISIN, SEDOL, CUSIP, SDCCUSIP to matching RICs. Then, it requests intrady summaries data with both RICs and plots a line char of each RICs's last trade prices.

## Pre-requisites

* Python 3.6 or later version.
* Required Python Packages: getpass, json, requests, pandas, numpy
* [Jupyter Notebook](https://jupyter.org/install). 
* EDP account with permission to access EDP symbology conversion and intraday summaries service.

# Implementation
The steps of implementation are the following:


__1. Enter EDP Username, Client Id and Password__

In [ ]:
import getpass as gp

username=input('Enter EDP username:')
clientid=input('Enter client id/app id:')
password=gp.getpass('Enter EDP Password:')

__2. Request access token with the inputs from step 1__

The application sends HTTP post with required parameters according to the [EDP tutorial](https://developers.refinitiv.com/elektron-data-platform/elektron-data-platform-apis/learning?content=38562&type=learning_material_item) to get the access token which is used to get data from EDP.

In [ ]:
from json import dumps, loads, load
from requests import post,get

#Get access token from EDP server 
getTokenEndpoint="https://api.refinitiv.com/auth/oauth2/beta1/token"
refreshToken=None
accessToken=None
_header= {}
_header['Accept']='application/json'
_params={}

if refreshToken is None:
    _params={
        "username": username,
        "password": password,
        "client_id": clientid,
        "grant_type": "password",
        "scope": "trapi",
        "takeExclusiveSignOnControl": "true"
    }
else:
    _params={
			"refresh_token": refreshToken,
			"username": username,
			"grant_type": "refresh_token"
    }

resp=post(url=getTokenEndpoint,data=_params,headers=_header,auth=(username,""))

__3. Verify a response and get access token in the successful response__

If the returned status code from HTTP response is not 200 (OK) that means the application gets an error response. Hence, it prints the response and exit. Otherwise, the application get the refresh token and access token from the HTTP response message and print them.

In [ ]:
refreshToken=""
accessToken=""
if resp.status_code!=200:
    print("Status Code:",resp.status_code," Text:",dumps(loads(resp.text),indent=4))
    exit()
else:
    from collections import OrderedDict
    auth_object=loads(resp.text,object_pairs_hook=OrderedDict)
    accessToken=auth_object["access_token"]
    refreshToken=auth_object["refresh_token"]

print("Access Token:",accessToken)
print()
print("Refresh Token:",refreshToken)

__4. Convert the identifiers to the RICs using Symbology Conversion Service__

The application sends a HTTP post with two input identifiers to Symbology Conversion Service. If the returned status code from HTTP response is not 200 (OK) that means the application gets an error response. Hence, it prints the response and exit. Otherwise, the application gets RICs from the HTTP response message and print the RICs. If all RICs matching the request are not returned, the program exits. 

In [ ]:
from requests import post,get
###Get data value(s) sorted acording to the names in headers key
def getDataByNameInHeaders(headersVal,dataVal,key):
    #check if data is null or empty
    if dataVal is None:
        return "There is no data"
    if(len(dataVal)==0):
        return "data is empty"
    position = -1
    dataList = []
    numHeaders = len(headersVal)
    #find position of key in name of headers
    for x in range (numHeaders): 
        nameHeader = headersVal[x]['name']
        if nameHeader == key:
            position = x
            break
    #if the key is not found
    if(position== -1):
        return key + " is not found"
    #if the key is found, get data
    numData = len(dataVal)
    for y in range (numData):
        data = dataVal[y][position]
        dataList.append(data)
    return dataList

#input two symbols e.g.ISIN, SEDOL, CUSIP, SDCCUSIP 
symbol1=input("Enter the first symbol: ")
symbol2=input("Enter the second symbol: ")
#keep dictionary which key is RIC and value is symbol
RIC2SymbolDict = {}

#create a request with the input symbols and send the request to Symbology Converion Service
symbology_conversion_endpoint="https://api.refinitiv.com/data/symbology/beta1/convert"
jsonreq={
    "universe": [
        symbol1,
        symbol2
    ],
    "to": [
        "RIC"
    ]
}
resp=post(url=symbology_conversion_endpoint,json=jsonreq,headers={"Authorization": "Bearer " + accessToken})
if resp.status_code!=200:
    print("Status Code:",resp.status_code," Text:",dumps(loads(resp.text),indent=4))
    exit()

sym_object=loads(resp.text)
print()
print("The OK response")
print(sym_object)
print()
#if the string is returned, it is error and there is no data so exit
instrumentList = getDataByNameInHeaders(sym_object['headers'],sym_object['data'],"instrument")
if isinstance(instrumentList, str):
    print("Error: " + instrumentList)
    exit()
RICList = getDataByNameInHeaders(sym_object['headers'],sym_object['data'],"RIC")
#Check if all RICs matching the request are returned. If not, the program exits.
getAllRICs = True
for x in range(len(instrumentList)):
    if RICList[x] is None:
        print(instrumentList[x] + " is failed to resolve to a RIC.")
        getAllRICs = False
    else:
        print(instrumentList[x] + " is RIC " + RICList[x])
        RIC2SymbolDict[RICList[x]]=instrumentList[x]
if getAllRICs == False:
    print("Some symbols cannot be converted to RICs. The program is exiting...");
    exit()

__5. Request intraday summaries data for each RIC__

The application sends a HTTP get to request data of the whole previous business day with interval 10 minutues in normal market session with the maximum is 25 for each RIC. If the returned status code from HTTP response is not 200 (OK) that means the application gets an error response. Hence, it prints the response and exit. Otherwise,it prints the successful response. If Requesting data of some RICs failed, the problem exits.
If you have not permission to request any RICs, please contact Refenitiv account team who can help you on this.

In [ ]:
#create parameters to intraday service
#calculate the previous business day which is start and end parameter
import pandas as pd
# BDay is business day
from pandas.tseries.offsets import BDay
import datetime
today = pd.datetime.today()
previousBDay=today-BDay(1)
historical_pricing_intraday_summaries_endpoint="https://api.refinitiv.com/data/historical-pricing/v1/views/intraday-summaries/"
separator="&"
interval="interval=PT10M"
startDate=str(previousBDay.date())+"T00:00:00.000000000Z"
start="start=" + startDate
endDate=str(previousBDay.date())+"T23:59:59.999999999Z"
end="end="+endDate
#Last trade price or value.
fields="fields=TRDPRC_1"
sessions="sessions=normal"
count="count=25"
params="?"+interval+separator+start+separator+end+separator+fields+separator+sessions+separator+count

#request data for each RIC
intraDayDataDictionary = {}
for aRIC in RICList:
    intradayUrl=historical_pricing_intraday_summaries_endpoint+aRIC+params
    print("Retrieve data of RIC " + aRIC)
    print("GET: "+intradayUrl)
    resp=get(url=intradayUrl,headers={"Authorization": "Bearer " + accessToken})
    
    if resp.status_code!=200:
        print("Status Code:",resp.status_code," Text:",dumps(loads(resp.text),indent=4))
        exit()
        break
    whole_intraday_object=loads(resp.text) 
    
    #Verify if there is data or not. If user has no permission, there is no data
    if 'data' not in whole_intraday_object[0]:
        print("There is no data")
        print("Failed response:")
        print(whole_intraday_object[0])
        break
    elif len(whole_intraday_object[0]['data'])==0:
        print("Data is empty")
        print("Failed response:")
        print(whole_intraday_object[0])
        break
    
    intraDayDataDictionary[aRIC]=whole_intraday_object[0]
    print("Successful response:")
    print(intraDayDataDictionary[aRIC])
    print()

__6. Prints data in the table format using pandas and numpy__

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame
dataFrameDictionary = {}
DateTimeDict = {}
dataToPlot= DataFrame()
#create table of each RIC

for aRIC in intraDayDataDictionary:
    intraday_object=intraDayDataDictionary[aRIC]
    headers=intraday_object['headers']
    #Get column headers/titles using lambda
    titles=map(lambda header:header['name'], headers)
    dataArray=np.array(intraday_object['data'])
    df=pd.DataFrame(data=dataArray,columns=titles)
    
    #if there is data
    if df.empty is False:
        df.index += 1
        dataFrameDictionary[aRIC]=df
        print("Data of " + aRIC + "(" + RIC2SymbolDict[aRIC] + ")")
        print(dataFrameDictionary[aRIC])
        print()
        #get date and time of the RIC
        DateTimeDict[aRIC]= getDataByNameInHeaders(intraday_object['headers'],intraday_object['data'],"DATE_TIME")

__7. Plot a line chart of each RICs__

Use matplotlib.pyplot module to plot a line chart of each RIC. The data for the X-axis is a date and time and the Y-axis is Last trade prices.

In [ ]:
import matplotlib.pyplot as plt
from pandas import DataFrame
import numpy as np
dataPlot= DataFrame()
yColumns = []

for aRIC in intraDayDataDictionary:
    dataFrameDictionary[aRIC].TRDPRC_1=dataFrameDictionary[aRIC].TRDPRC_1.astype(float)
    df=dataFrameDictionary[aRIC]
    #get data rows which 'TRDPRC_1' field has value
    df=df[np.isfinite(df['TRDPRC_1'])]
    #set size and grid of chart
    fig = plt.figure(figsize=(6,3),dpi=100)
    ax = plt.subplot2grid((3,3), (0, 0), rowspan=3, colspan=3)
    ax.grid(True)
    #set title and label of x and y axis
    ax.set_title("Last Trade Prices of "+ aRIC + "(" + RIC2SymbolDict[aRIC] + ")")
    ax.set_xlabel('Date and Time')
    plt.xticks(rotation=90)
    ax.set_ylabel('Price')
    #plot a line chart with x axis is 'Date-Time' field and y axis is 'TRDPRC_1' field
    ax.plot(df['DATE_TIME'], df['TRDPRC_1'])
    #show the line chart
    plt.show()